In [77]:
from selenium import webdriver
import bs4
import pandas as pd

# 셀레니움을 사용하여 HTML태그를 반환하는 함수입니다.
def get_sources():
    
    sources = []
    
    driver = webdriver.Chrome('C:\웹 드라이버\chromedriver.exe')
    driver.implicitly_wait(1)
    driver.get('https://www.jdcenter.com/prcenter/issue/notice.cs')
    
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'html.parser')
    trs = soup.find_all("tr")

    # 내부에 존재하는 <tr>태그를 탐색합니다.
    for tr in trs:
        tds = tr.find_all('td')
        
        # <tr>태그 내부에 존재하는 <td>태그를 리스트에 삽입합니다.
        for td in tds:
            source = td.text
            sources.append(source)
            
    return sources


In [79]:
sources = get_sources()

In [80]:
# 제목, 작성자, 작성일만 파싱하여 새로운 리스트에 추가합니다.
name_data = []
offeror_data = []
date_data = []

for i in range(len(sources)):
    if i % 6 == 1:
        name_data.append(sources[i])
    elif i % 6 == 3:
        offeror_data.append(sources[i])
    elif i % 6 == 4:
        date_data.append(sources[i])

In [97]:
dict_list = {'name' : name_data, 'offeror' : offeror_data, 'date' : date_data}

df = pd.DataFrame(dict_list)
df.to_csv('data_set.csv')